In [1]:
# Are UK families well fed? What predicts it?
# Lucas Gonzalez Santa Cruz - Data Science Intensive - Sept 2016

# ------------- O U T L I N E ---------------
# - Introduction
# - Get the files from the web
# - Households
# - Diaries
# - Nutrients
# - Score / Classification
# - Models

In [2]:
# ------------- I N T R O D U C T I O N ---------------

In [3]:
# DATA:
# UK families (each with adult males and females, and children) wrote down what food they bought over a week.
# - 6000 families (with socio-economic descriptors).
# - 315 foods (each with quantities and 47 nutrients) in 18 food-groups.
# === 1974-2000:
# visualisation: http://britains-diet.labs.theodi.org/
# overview: http://theodi.org/blog/family-food-publication-day-blog-feeding-the-hunger-for-data
# survey methodology: https://www.gov.uk/government/publications/family-food-methodology
# === 2001-etc:
# not open data: https://www.gov.uk/government/statistical-data-sets/family-food-datasets

# GOALS:
# Compute a nutritional "score" (higher is better) or "category" (well/badly).
# Build a regression/classification model to look for predictive or explanatory variables.

# SCORE (CAN BE CATEGORISED):
# Ratio between (a) calories bought per person and per day
# and (b) recommended calories for each family (say one female, one male, and 2 children).
# (When "calories" are done, "protein" etc can be tried, and more complex models built.)



In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
data2000 = 'data/NFSopen_2000/'
dataRefe = 'data/NFSopen_Reference/'
pd.options.display.width = 180

In [5]:
# ------------- H O U S E H O L D S ---------------

In [6]:
#In 2000, 53 variables from 6699 households were collected.
df_house2000 = pd.DataFrame.from_csv(data2000+'2000 household data.txt', sep='\t', index_col=None)
#df_house2000.shape #(6699, 53)
# 'hhno', 'gormet2', 'reg', 'lad', 'styr', 'stmth', 'mic', 'frez', 'owndw', 'memhh',
# 'schmilk', 'incgp', 'occhoh', 'sochoh', 'dacthoh', 'Finc_decile_by_members_of_hh',
# 'finctp', 'pernohoh', 'pernohw', 'agehoh_banded', 'agehw_banded', 'adltm', 'adltf',
# 'child', 'oaps', 'adltgt64', 'stqtr', 'country', 'gor', 'agemdk_banded', 'incgpa',
# 'hhcomp', 'hhcompa', 'hcxigs', 'dat1rec', 'doormilk', 'frij', 'earners', 'occxhoh',
# 'socxhoh', 'dactxhoh', 'szwkest', 'empst', 'wkdy1rec', 'vegind', 'gornw', 'pernxhoh',
# 'jobsthoh', 'incgp745', 'withjob', 'landlord', 'furnish', 'benefits'

#This is the description of each of the 53 fields:
df_house_fields = pd.DataFrame.from_csv(dataRefe+'house-fields.txt', sep='\t', index_col=None)
df_house_fields.columns=['table', 'field', 'field_text', 'type1', 'size', 'type2', 'lookup', 'notes']
#df_house_fields.drop(['table', 'type1', 'size', 'type2'], inplace=True, axis=1)
df_house_fields = df_house_fields[['field', 'field_text', 'lookup', 'notes']]
#df_house_fields.head()
#df_house_fields.shape #(53, 4)

#Family composition:
#hhno: household number
#adltm: number of male adults (18 years or over plus 16 or 17 head of households)
#adltf: number of female adults (18 years or over plus 16 or 17 head of households)
#child: number of children
df_house = df_house2000[['hhno', 'adltm', 'adltf', 'child']]
df_house.columns=['house', 'men', 'women', 'children']
#df_house.head()

#Calories recommended:
# http://www.foodlabel.org.uk/label/gda_values.aspx#item1
# Guideline Daily Amount Values Typical values
# Women 	Men 	Children (5-10 years)
# Calories 	2,000 kcal 	2,500 kcal 	1,800 kcal
# There are also recommendations for Protein, Carbohydrate, Sugars, Fat, Saturates, Fibre and Salt
df_house['calories_rec'] = (df_house.women * 2000) +  (df_house.men * 2500) + (df_house.children * 1800)
df_house.head(5)

/home/lucas/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,house,men,women,children,calories_rec
0,261117,2,1,0,7000
1,261118,1,0,0,2500
2,261119,1,0,0,2500
3,261120,1,1,3,9900
4,261121,0,1,0,2000


In [7]:
# Many of the household fields have auxiliary files with (code, text).

# I wrote this to read the auxiliary files:
"""
#df_matrix = df_house_fields.as_matrix()
#for line in df_matrix:
#    #print line[0], line[2]
#    #print "*"+str(line[2])+"*"
#    if str(line[2]) != "nan" :
#        filename = str(line[2])
#        filename = filename.replace(":", "_")
#        print "#" + line[1]
#        print "df_" + line[0] + " = pd.DataFrame.from_csv(dataRefe+'" + filename + "', sep='\\t', index_col=None)"
#        print "#df_" + line[0] + ".columns=['','']"
#        print "#df_" + line[0] + ".shape"
#        print "#df_" + line[0] + ".head(5)"
#        print
"""

# Then, I fixed the resulting code (below): uncommented some fields, fixed some by hand.

#government office region metropolitan county = 27 x (code, text)
#df_gormet2 = pd.DataFrame.from_csv(dataRefe+'DCV_ GORMET2.txt', sep='\t', index_col=None)
#df_gormet2.columns=['gormet', 'gormet_text']

#standard statistical region = 11 x (code, text); Scotland = 1
df_reg = pd.DataFrame.from_csv(dataRefe+'DCV_ Region.txt', sep='\t', index_col=None)
df_reg.columns=['region','region_text']

#local authority district = 457 x (code, text)
#df_lad = pd.DataFrame.from_csv(dataRefe+'DCV_ LAD.txt', sep='\t', index_col=None)
#df_lad.columns=['lad','lad_text']

#ownership of microwave (1 = yes 0= not)
df_mic = pd.DataFrame.from_csv(dataRefe+'DCV_ Microwave ownership.txt', sep='\t', index_col=None)
df_mic.columns=['microwave','microwave_text']

#ownership of freezer (1 = yes 0= not)
df_frez = pd.DataFrame.from_csv(dataRefe+'DCV_ Freezer ownership.txt', sep='\t', index_col=None)
df_frez.columns=['freezer','freezer_text']

#tenure (ownership of dwelling) = 7 categories
df_owndw = pd.DataFrame.from_csv(dataRefe+'DCV_ Ownership of dwelling.txt', sep='\t', index_col=None)
df_owndw.columns=['own_dwelling','own_dwelling_text']

#school milk #yes, no
df_schmilk = pd.DataFrame.from_csv(dataRefe+'DCV_ School milk.txt', sep='\t', index_col=None)
df_schmilk.columns=['school_milk','school_milk_text']

#income group of the head of the household = 8 categories
#df_incgp = pd.DataFrame.from_csv(dataRefe+'DCV_ Income Group.txt', sep='\t', index_col=None)

#occupation of the head of the household = 193 categories, 3 columns
#df_occhoh = pd.DataFrame.from_csv(dataRefe+'DCV_ OCC_OCCHOH Occupation class with unpacked descriptions.txt', sep='\t', index_col=None)

#social occupation class of the head of the household = 380 categories
#df_sochoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Standard occupation class.txt', sep='\t', index_col=None)
#df_sochoh.columns=['hhh_job','hhh_job_text']

#degree of activity of the head of the household = sedentary, active, moderate
df_dacthoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Degree of activity.txt', sep='\t', index_col=None)
df_dacthoh.columns=['head_sedentary','head_sedentary_text']

#family income type #how we know about income: net, gross, estimated
#df_finctp = pd.DataFrame.from_csv(dataRefe+'DCV_ Family income tp.txt', sep='\t', index_col=None)
#df_finctp.columns=['finctp','finctp_text']

#age of head of household (banded) = 7 age groups
df_agehoh_banded = pd.DataFrame.from_csv(dataRefe+'Ref_Age_bands_for_household_table.txt', sep='\t', index_col=None)
df_agehoh_banded.columns=['age_headhouse','age_headhouse_text']

#age of housewife (banded) = 7 age groups
df_agehw_banded = pd.DataFrame.from_csv(dataRefe+'Ref_Age_bands_for_household_table.txt', sep='\t', index_col=None)
df_agehw_banded.columns=['age_housewife','age_housewife_text']

#government office region = 13 regions, Scotland is 12
df_gor = pd.DataFrame.from_csv(dataRefe+'DCV_ GOR.txt', sep='\t', index_col=None)
df_gor.columns=['gor','gor_text']

#age of main diary keeper = 7 age groups
#df_agemdk_banded = pd.DataFrame.from_csv(dataRefe+'Ref_Age_bands_for_household_table.txt', sep='\t', index_col=None)

#income group (1 = All A  2=B  3=C 4=D & E2 5=E1 & OAPs)
#df_incgpa = pd.DataFrame.from_csv(dataRefe+'DCV_ Income group (A group).txt', sep='\t', index_col=None)
#df_incgpa.columns=['income_group','income_group_text']

#household composition = number of adults (male and female) and children
df_hhcomp = pd.DataFrame.from_csv(dataRefe+'DCV_ Household Composition.txt', sep='\t', index_col=None)
df_hhcomp.columns=['hhcomp','hhcomp_text', 'hhcomp_text_long']

#household composition (all adults) = number of adults and children - NOT USEFUL IMHO
#df_hhcompa = pd.DataFrame.from_csv(dataRefe+'DCV_ Household Compositon (A group).txt', sep='\t', index_col=None)
#df_hhcompa.columns=['adults_children','adults_children_text']

#household composition by income group = 28 categories, how many family members are this or that
#df_hcxigs = pd.DataFrame.from_csv(dataRefe+'DCV_ Household composition x income group.txt', sep='\t', index_col=None)
#df_hcxigs.columns=['hh_comp_income','hh_comp_income_text']

#doorstep milk delivery (1 = yes 2 = no)
df_doormilk = pd.DataFrame.from_csv(dataRefe+'DCV_ Doormilk.txt', sep='\t', index_col=None)
df_doormilk.columns=['doormilk','doormilk_text']

#ownership of fridge  (1 = yes 0= not)
df_frij = pd.DataFrame.from_csv(dataRefe+'DCV_ Fridge ownership.txt', sep='\t', index_col=None)
df_frij.columns=['fridge','fridge_text']

#occupation of crossover head of household
#df_occxhoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Occupation class.txt', sep='\t', index_col=None)
#df_occxhoh.columns=['occupation_class','occupation_class_text']

#social occupation class of cross head of household #kinds of job
#df_socxhoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Standard occupation class.txt', sep='\t', index_col=None)
#df_socxhoh.columns=['socxhoh','socxhoh_text']

#degree of activity of cross over head of household #sedentary etc
#df_dactxhoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Degree of activity.txt', sep='\t', index_col=None)
#df_dactxhoh.columns=['deg_activity','deg_activity_text']

#size of work establishment of head of household #sme etc
#df_szwkest = pd.DataFrame.from_csv(dataRefe+'DCV_ Size of work establishment.txt', sep='\t', index_col=None)
#df_szwkest.columns=['hoh_job_size','hoh_job_size_text']

#employment status of head of household (1 = employed 2 = unemployed)
#df_empst = pd.DataFrame.from_csv(dataRefe+'DCV_ Employment status.txt', sep='\t', index_col=None)
#df_empst.columns=['hoh_employed','hoh_employed_text']

#weekday (1 st day of recording) 1 = Sunday....7=Saturday
#df_wkdy1rec = pd.DataFrame.from_csv(dataRefe+'DCV_ Weekday (1st day of recording).txt', sep='\t', index_col=None)
#df_wkdy1rec.columns=['weekday','weekday_text']

#vegetarian indicator
df_vegind = pd.DataFrame.from_csv(dataRefe+'DCV_ Vegetarian indicator.txt', sep='\t', index_col=None)
df_vegind.columns=['vegetarian','vegetarian_text']

#government office region for northwest and merseyside (1=north west/merseyside 2=other)
#df_gornw = pd.DataFrame.from_csv(dataRefe+'DCV_ GORNW.txt', sep='\t', index_col=None)
#FILE DOES NOT EXIST -- CHECK

#job status of head of household
df_jobsthoh = pd.DataFrame.from_csv(dataRefe+'DCV_ Job status of HOH.txt', sep='\t', index_col=None)
df_jobsthoh.columns=['hohjobstatus','hohjobstatus_text']

#income group (1974-1975)
#df_incgp745 = pd.DataFrame.from_csv(dataRefe+'DCV_ Income group 7475.txt', sep='\t', index_col=None)
#df_incgp745.columns=['income_group','income_group_text']

#with job indicator - accomodation tied to employment
df_withjob = pd.DataFrame.from_csv(dataRefe+'DCV_ WITHJOB (1998 to 2000 only).txt', sep='\t', index_col=None)
df_withjob.columns=['withjob','withjob_text']

#landlord status indicator: type of landlord
#df_landlord = pd.DataFrame.from_csv(dataRefe+'DCV_ LANDLORD (1998 to 2000 only).txt', sep='\t', index_col=None)
#df_landlord.columns=['landlord','landlord_text']

#furnished indicator: is the accomodation furnished
#df_furnish = pd.DataFrame.from_csv(dataRefe+'DCV_ FURNISHED (1998 to 2000 only).txt', sep='\t', index_col=None)
#FILE DOES NOT EXIST -- CHECK

In [8]:
# ------------- D I A R I E S : FOOD BOUGHT BY THE HOUSEHOLDS DURING ONE WEEK ---------------

In [9]:
#6k+ families recorded their food purchases for a week. One line per food-item: 3 pints of milk, etc.
df_diary = pd.DataFrame.from_csv(data2000+'2000 diary data.txt', sep='\t', index_col=None)
df_diary.columns=['hhno', 'fooditem', 'logday', 'purchasevalue', 'minor', 'quantity', 'purchasefree']
#hhno = household number
#fooditem = food item number: if on a given day you buy bread and milk, one would be 1 and the other 2.
#logday (1 equals the first day of record keeping)
#purchase value (pence)
#minor = 315 detailed food codes
#quantity: available for consumption (includes bones etc, which are not consumed).
#          In imperial numbers except confectionery and alcohol, in metric units 1992-2000.
#purchasefree: "1,5 & 9 indicate purchased food 2, 3, 4, 6, 7 & 8 indicate free food."
df_diary = df_diary[['hhno', 'minor', 'quantity']]

# 315 detailed "minor" foodcodes --- then aggregated into 183 "major" groups.
df_min_maj = pd.DataFrame.from_csv(dataRefe+'Ref_ Minor and major foods.txt', sep='\t', index_col=None)
df_min_maj.columns=['minor', 'minor_text', 'major']
df_maj_text = pd.DataFrame.from_csv(dataRefe+'Ref_ Major food codes.txt', sep='\t', index_col=None)
df_maj_text.columns=['major', 'major_text']
#Each "foodcode" was described with one of 7 "units": pints, ounces, etc.
df_min_units = pd.DataFrame.from_csv(dataRefe+'Ref_MINFD_Minor_food_codes.txt', sep='\t', index_col=None)
df_min_units.columns=['minor','minor_text','units']
df_min_units.drop(['minor_text'], inplace=True, axis=1) #to avoid duplicate later

#24 more aggregated "groups" were defined, and the 183 previous "detailed groups" were mapped to these 24.
df_grp_text = pd.DataFrame.from_csv(dataRefe+'Ref_ food groups (standard).txt', sep='\t', index_col=None)
df_grp_text.columns=['group','group_text']
df_mapping = pd.DataFrame.from_csv(dataRefe+'Ref_ Major-food group mapping.txt', sep='\t', index_col=None)
df_mapping.columns=['major','group']
#The mapping included 92 groups, and those 24 had to be selected.
group24 = [4006, 9017, 22023, 31041, 46094, 100127, 129129, 135148, 150154, 155161, 162171, 172183, \
           184208, 210231, 233248, 251263, 264264, 267277, 281301, 304313, 314339, 340344, 350354, 380389]
df_maj_group = df_mapping[df_mapping['group'].isin(group24) == True]

#Food diaries were merged into one table:
df = pd.merge(df_diary, df_min_maj, how='left', on='minor')
df = pd.merge(df, df_min_units, how='left', on='minor')
df = pd.merge(df, df_maj_text, how='left', on='major')
df = pd.merge(df, df_maj_group, how='left', on='major')
df_diary = pd.merge(df, df_grp_text, how='left', on='group')
df_diary = df_diary[['hhno', 'quantity', 'units', 'minor_text', 'group_text', 'minor', 'major', 'major_text', 'group']]
print "family - qty, units, food, group) - details:"
df_diary.head(3)

family - qty, units, food, group) - details:


,hhno,quantity,units,minor_text,group_text,minor,major,major_text,group
0,261119,6.77,oz,SOUPS DEHYDRATED & POWDERED,ALL OTHER FOODS,31901,319,SOUPS DEHYDRATED AND POWDERED,314339
1,261119,28.21,oz,BREAD WHOLEMEAL SLICED,ALL BREAD,26001,260,BREAD WHOLEMEAL SLICED,251263
2,261119,1.75,pt,SEMI AND OTHER SKIMMED MILKS,OTHER MILK & CREAM,1503,15,SKIMMED MILKS,9017


In [10]:
# ------------- N U T R I E N T S ---------------

In [11]:
#We have the 47 nutrients (txt from a PDF)
df_nut_names = pd.DataFrame.from_csv(dataRefe+'Nutrient Conversion factors.txt', sep=',', index_col=None)
df_nut_names.columns=['nutrient','nutr_text', 'nutr_comments']
df_nut_names.head(8) # we're interested in nutrient = 8 (Energy - Kcal)
#We have the conversion factors for each 315 'minor', for each of the 4 quarters of year 2000
df_nut2000 = pd.DataFrame.from_csv(data2000+'2000 nutrient conversion factors.txt', sep='\t', index_col=None)
df_nut2000.columns=['minor', 'nutrient', 'year', 'quarter', 'nutconvfactor']
#We merge the nutrients with the conversion factors
df_nut = pd.merge(df_nut2000, df_nut_names, how='left', on='nutrient')
df_nut = pd.merge(df_nut, df_min_maj, how='left', on='minor')
df_nut.drop(['major'], inplace=True, axis=1) #to avoid duplicate later
#.shape #(43348, 8) (some of the 'minor' had no nutrients)
#'minor', 'nutrient', 'year', 'quarter', 'nutconvfactor', 'nutr_text', 'nutr_comments', 'minor_text'
print "Notice how the NCV is different for each quarter (of 2000), so later we'll have to average them"
df_nut[['minor_text', 'nutr_text', 'quarter', 'nutconvfactor']].head(9)

Notice how the NCV is different for each quarter (of 2000), so later we'll have to average them


,minor_text,nutr_text,quarter,nutconvfactor
0,MILK LIQUID FULL PRICE,Animal Protein,1,17.00
1,MILK LIQUID FULL PRICE,Animal Protein,2,19.00
2,MILK LIQUID FULL PRICE,Animal Protein,3,20.00
3,MILK LIQUID FULL PRICE,Animal Protein,4,19.00
4,MILK LIQUID FULL PRICE,Fat,1,22.00
5,MILK LIQUID FULL PRICE,Fat,2,21.00
6,MILK LIQUID FULL PRICE,Fat,3,22.00
7,MILK LIQUID FULL PRICE,Fat,4,23.00
8,MILK LIQUID FULL PRICE,Saturates,1,13.89


In [12]:
df = df_nut[df_nut.nutrient == 8] #Select nutrient = Calories
df = df.pivot(index='minor', columns='quarter', values='nutconvfactor')
df.columns.name = None
df.columns = ['q1', 'q2', 'q3', 'q4']
df['calories'] = (df.q1 + df.q2 + df.q3 + df.q4) / 4 #average all the calories for the 4 quarters of 2000
df_min_cal = df
df_min_cal.head(3) #307, not 315

,q1,q2,q3,q4,calories
minor,,,,,
401,380.00,370.00,380.0,390.0,380.0000
402,404.38,374.53,382.8,399.8,390.3775
403,404.38,374.53,382.8,399.8,390.3775


In [13]:
#read foods
df_min_maj = pd.DataFrame.from_csv(dataRefe+'Ref_ Minor and major foods.txt', sep='\t', index_col=None)
df_min_maj.columns=['minor', 'minor_text', 'major']
df_food = df_min_maj
df_food.index = df_food['minor']
#read calories for each food
df_min_cal.index = df_min_cal['minor']
df_cal = df_min_cal[['minor', 'calories']]
#merge
df_food_cal = pd.merge(df_food, df_cal, how='left', on='minor')
df_food.head(55)

KeyError: 'minor'

In [ ]:
df = pd.merge(df3, df_diary, how='left', on='minor')
df[['minor', 'quantity', 'calories']].head(5)
df_cal_consumed = df
df_cal_consumed['calconsumed'] = df_cal_consumed.quantity * df_cal_consumed.calories
df_cal_consumed[['minor', 'quantity', 'calories', 'calconsumed']].head(5)
df_cal_consumed.shape

In [ ]:
# ------------- S C O R E / C L A S S I F I C A T I O N ---------------

In [ ]:
#df_cal_consumed has the calories consumed in detail: by hh and by fooditem
#i'd like to sum that up into df_cal_household, with all the calories consumed by each family
#currently i have no idea how to do that ... groupby then aggregate ?????

import numpy as np
df_cal_household = df_cal_consumed.groupby('hhno')
print len(df_cal_consumed)
print len(df_cal_household)
df_cal_household.aggregate(np.sum) # DOESN'T WORK !!! TIME FOR A REST !!!
#df_cal_household.groupby('calories', as_index=False).sum()

sum_cal = df_cal_consumed.groupby('hhno')['calories'].sum()
sum_cal.head()

In [ ]:
df_cal_rec.head()

In [ ]:
#now i only have to join them!

In [ ]:
# ------------- M O D E L S ---------------

In [ ]:
# ------------- R A N D O M   S T U F F   A N D   E X P E R I M E N T S ---------------

In [ ]:
#a) convert units to grams
#b) select the number of family members
#c) calculate "grams of each food-group _per person_" for each broad age-group
#d) Look at the cells below to re-sort the food-group categories
#e) Find how to add extra text that sits outside the cells
#f) Re-write the project

In [ ]:
num_houses = len(df_house2000)
print "Num of households: ", num_houses
num_scotland = len(df_house2000[df_house2000.reg == 1])
print "Num of households in Scotland: ", num_scotland
print "Proportion of questionaires from Scotland: ", (num_scotland*100) / num_houses, "%"

In [ ]:
df_hhcomp.columns=['hhcomp', 'hhcomp_text', 'hhcomp_text_long']

In [ ]:
df_age_comp = pd.merge(df_house2000, df_hhcomp, how='left', on='hhcomp')
df_age_comp = df_age_comp[['hhno', 'hhcomp', 'hhcomp_text', 'hhcomp_text_long']]

#i ___thought___ i need to create variables "adults", "children" and then
#if hh_comp = 1 then adult = 1, children = 0, etc
#df_hhcomp #this 2, 7, 9, 10 and 11 are indeterminate - how many are like this?

df_indeterminate = df_age_comp[df_age_comp.hhcomp.isin([2, 7, 9, 10, 11])]
#df_indeterminate.shape #(51498, 4)
print (len(df_indeterminate)*100/len(df_age_comp)), "% of households (", \
       len(df_indeterminate), "of", len(df_age_comp), ") have an indeterminate number of people (x or more)", 

In [ ]:
test=df_age_comp.groupby('hhcomp_text_long')
test.size()

In [ ]:
test=df_indeterminate.groupby('hhcomp_text_long')
test.size()

In [ ]:
#So what could we assume?
#In theory:
#A=1, C=? ... C=average number of children in families with A=1 ... for convenience C=1
#A=2, C>3 ... C=average number of children in families with A=2 and C=4, 5, etc ... for convenience C=4
#A>2, C=1or2 ... A=average number of adults in families with C=1or2, C=average number of children in families with C=1or2
#A>2, C>2 ... A=average number of adults in families with C=3, 4, etc; C=average number of children in families with A=3, 4, etc
#A>3 ... A=average number of adults in families with C=0
#But in practice we don't have those details in our data. So, could we have that from national statistics?
#Number of children in monoparental families in the UK ...
#http://webarchive.nationalarchives.gov.uk/20160105160709/http://www.ons.gov.uk/ons/rel/family-demography/family-size/2012/family-size-rpt.html
#For convenience (real people will be that or more -> real food/person will be _slightly lower_ than our estimates):
#A=1, C=? ... C=1
#A=2, C>3 ... C=4
#A>2, C=1or2 ... A=3
#A>2, C>2 ... A=3, C=3
#A>3 ... A=4

#import numpy as np
#dftest = df
#dftest['adults'] = np.where(dftest['hhcomp']==1, 1, 0)
#dftest.head(5)
#['adults', 'children'] #add variables?

def get_adults (row): #2531
    if row['hhcomp'] == 1:
        return 1
    if row['hhcomp'] == 2 :
        return 1
    if row['hhcomp'] == 3 :
        return 2
    if row['hhcomp'] == 4 :
        return 2
    if row['hhcomp'] == 5 :
        return 2
    if row['hhcomp'] == 6 :
        return 2
    if row['hhcomp'] == 7 :
        return 2
    if row['hhcomp'] == 8 :
        return 3
    if row['hhcomp'] == 9 :
        return 3
    if row['hhcomp'] == 10 :
        return 3
    if row['hhcomp'] == 11 :
        return 4
df_age_comp.apply (lambda row: get_adults (row), axis=1)
df_age_comp['adults'] = df_age_comp.apply (lambda row: get_adults (row),axis=1)

def get_children (row): #01012340130
    if row['hhcomp'] == 1:
        return 0
    if row['hhcomp'] == 2 :
        return 1
    if row['hhcomp'] == 3 :
        return 0
    if row['hhcomp'] == 4 :
        return 1
    if row['hhcomp'] == 5 :
        return 2
    if row['hhcomp'] == 6 :
        return 3
    if row['hhcomp'] == 7 :
        return 4
    if row['hhcomp'] == 8 :
        return 0
    if row['hhcomp'] == 9 :
        return 1
    if row['hhcomp'] == 10 :
        return 1.5
    if row['hhcomp'] == 11 :
        return 0
df_age_comp.apply (lambda row: get_children (row), axis=1)

df_age_comp['children'] = df_age_comp.apply (lambda row: get_children (row),axis=1)

df_age_comp.head(5)

In [ ]:
df_diary_with_ages = pd.merge(df_diary, df_age_comp, how='left', on='hhno')
df_diary_with_ages.head(3)


In [ ]:
#pendiente de revisar ... y falta ver lo de las edades bien visto!!!
df.columns

In [ ]:
df2 = df
df2.drop(['logday', 'purchasevalue', 'minor', 'purchasefree', 'major', 'group', 'hhcomp', 'hhcomp_text', 'hhcomp_text_long'], inplace=True, axis=1)
df2.columns

In [ ]:
print "Total number of adults: ", df2.adults.sum()
print "Total number of children: ", df2.children.sum()
print "Average number of adults per household: ", (df2.adults.sum()*1.0)/len(df_diary)
print "Average number of children per household: ", (df2.children.sum()*1.0)/len(df_diary)
print "Average number of people per household: ", (df2.adults.sum()*1.0)/len(df_diary) + (df2.children.sum()*1.0)/len(df_diary) 

In [ ]:
#----- TIME TO FOOL AROUND!!! How much of each of the 24 groups in 2000? <====================

In [ ]:
# http://britains-diet.labs.theodi.org/ "explore the data", "overview", "2000".

In [ ]:
num_of_households = len(df.household.unique())
num_of_households

In [ ]:
sumfreq = df.groupby(by=['group_text'])['quantity'].sum() / num_of_households
#But some households have more people than others
#And foods have to be converted to 'grams per week' before they can be added
#Or even to 'grams per day' if we are to compare with the Canaries
#And groups should be sorted in the same way as in britains-diet
df_sumfreq = sumfreq.to_frame()
df_sumfreq.sort_index()

In [ ]:
"""
POTATOES
FRESH GREEN VEGETABLES
OTHER FRESH VEGETABLES
ALL PROCESSED VEGETABLES
ALL BREAD
FRESH FRUIT
FRUIT & FRUIT PRODS. NOT FRESH
ALL NON-CARCASE MEAT AND MEAT PRODUCTS
SUGAR AND PRESERVES
ALL CARCASE MEAT
ALL FATS
ALL OTHER FOODS
CEREALS, EXCL. BREAD,BUNS,CAKES,BISCUITS
BISCUITS, CAKES, BUNS, CRISPBREADS
FLOUR
ALL FISH
BEVERAGES
TOTAL CHEESE
CONFECTIONERY
---
LIQUID WHOLEMILK, INC SCHOOL & WELFARE
OTHER MILK & CREAM
SOFT DRINKS
ALCOHOLIC DRINKS
...rest not sorted yet...
EGGS
"""

In [ ]:
df_sumfreq = df_sumfreq.sort_values(by='quantity', ascending=False) #.head(20)
my_plot = df_sumfreq.sort_values(by='quantity',ascending=True).plot(kind='barh',legend=None,title="Total Quantities by Food Category")
my_plot.set_xlabel("Foods")
my_plot.set_ylabel("Quantities")

In [ ]:
#----- OLDER ATTEMPTS WHICH MIGHT BE HELPFUL NOW I HAVE ONE GOOD TABLE

In [ ]:
#df.minor_text.unique()

In [ ]:
#df_merged.describe()

In [ ]:
#sumfreq = df_merged.groupby(by=['foodcat'])['qty'].sum()
#df_sumfreq = sumfreq.to_frame()
#df_sumfreq.sort_index()

In [ ]:
#df_sumfreq = df_sumfreq.sort_values(by='qty', ascending=False).head(20)
#my_plot = df_sumfreq.sort_values(by='qty',ascending=True).plot(kind='barh',legend=None,title="Total Quantities by Food Category")
#my_plot.set_xlabel("Foods")
#my_plot.set_ylabel("Quantities")

In [ ]:
#pd.crosstab(index=df_merged["minfddesc"], columns='cq') #gives us all the food items with a count (not a *sum* of cq!)

In [ ]:
#df_merged[df_merged.minfddesc == 'YOGHURT'] #3838 purchases of yoghurt, each with their own cq

In [ ]:
#df_merged[df_merged.minfddesc == 'SEMI AND OTHER SKIMMED MILKS'] #10898 purchases of SEMI AND OTHER SKIMMED MILKS,
                                                                # each with their own cq

In [ ]:
#now, add all cq's ...

#this tells us how many purchases of each food
#df_merged.minfddesc.value_counts() #10898 purchases of milk

#this tells us how many purchases were of each quantity
#df_merged[df_merged.minfddesc == 'SEMI AND OTHER SKIMMED MILKS'].cq.value_counts() #3678 purchases of milk were 1 (pint?)
#df_merged[df_merged.minfddesc == 'YOGHURT'].cq.value_counts() #710 purchases of yoghurt were 0.87

#but i'd like to say YOGHURT = so many grams in total, and I still can't do that
#in Epiinfo it wouldbe SUMFREQ CQ MINFDDESC == each value of MINFDDESC followed by the sum of grams for that value
#that's my task for my next 2 hour chunk

In [ ]:
#major food description -> horizontal bar chart

In [ ]:
#http://pandas.pydata.org/pandas-docs/stable/visualization.html
#http://pandas.pydata.org/pandas-docs/stable/groupby.html
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
#http://pbpython.com/simple-graphing-pandas.html <--- useful!
#http://www.gallamine.com/2014/07/python-pandas-group-by-column-and-sum.html <--- useful!
#http://hamelg.blogspot.co.uk/2015/11/python-for-data-analysis-part-19_17.html <--- useful!
#http://dataconomy.com/14-best-python-pandas-features/
#https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/
#https://www.analyticsvidhya.com/blog/2015/04/comprehensive-guide-data-exploration-sas-using-python-numpy-scipy-matplotlib-pandas/
#http://machinelearningmastery.com/quick-and-dirty-data-analysis-with-pandas/
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html

In [ ]:
"""
from __future__ import division
import time

import matplotlib.pyplot as plt
import pandas as pd
import nltk
import numpy as np

from PIL import Image

%matplotlib inline

# some nicer colors from http://colorbrewer2.org/
COLOR1 = '#7fc97f'
COLOR2 = '#beaed4'
COLOR3 = '#fdc086'
COLOR4 = '#ffff99'
COLOR5 = '#386cb0'

color_list = [COLOR1, COLOR2, COLOR3, COLOR4, COLOR5]
"""

In [ ]:
"""
merged[merged.minfddesc.str.startswith('YO')].head()
grupos = merged.groupby('minfddesc')

merged.sort_values('cq', ascending = False)

sums = merged.minfddesc.value_counts()
sums

totalc = len(merged)
series_minfddesc = merged.minfddesc.value_counts() / totalc * 100
df_minfddesc = series_minfddesc.to_frame(name=None)
df_minfddesc.shape
#df_minfddesc.head()

df = df_minfddesc.sort_index()
df.plot(title='Relative Frequency Distribution of minfddesc', kind='barh', color=COLOR5, grid=True)
df.plot()

#slowly getting somewhere, but i need a mentor for the python language
#it cannot be that doing a FREQ VARIABLE is so darn difficult
pd.crosstab(index=df["minfddesc"],  # Make a crosstab
                              columns="minfddesc")      # Name the count column
"""